In [1]:
import torch
import torchvision
import torch.nn as nn
from model1 import resnet34
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F
from resnetself import resnet50

In [2]:
def get_num_correct(preds,labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [3]:
train_set = torchvision.datasets.CIFAR10(
    root='../data/CIFAR10/'
    ,train=True
    ,download=False
    ,transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip()        
    ])
)

In [7]:

import os
import torch
print(torch.cuda.is_available())#是否有可用的gpu
print(torch.cuda.device_count())#有几个可用的gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"#声明gpu
dev=torch.device('cuda:0')#调用哪个gpu
a=torch.rand(100,100).to(dev)
torch.cuda.empty_cache()
print(torch.__version__)
print(torch.version.cuda)

True
1
1.11.0
11.3


In [4]:
network = resnet50(num_classes = 10).to('cuda')
network.train()

train_loader = torch.utils.data.DataLoader(train_set,batch_size = 36)
optimizer = optim.Adam(network.parameters(),lr = 0.0001)

batch = next(iter(train_loader))
images = batch[0].to('cuda')
labels = batch[1].to('cuda')

preds = network(images)
loss = F.cross_entropy(preds,labels)

loss.backward()#计算梯度
optimizer.step()#更新权重

#-----------------------
print('loss1:',loss.item())
preds = network(images)
loss = F.cross_entropy(preds,labels)
print('loss2:',loss.item())


loss1: 7.240841388702393
loss2: 6.54008674621582


In [4]:
network = resnet50(num_classes = 10).to('cuda')
network.train()

train_loader = torch.utils.data.DataLoader(train_set,batch_size=36)
optimizer = optim.Adam(network.parameters(),lr = 0.0001)

total_loss = 0
total_correct = 0
count = 0
for batch in train_loader:
    images = batch[0].to('cuda')
    labels = batch[1].to('cuda')
    
    preds = network(images)
    loss = F.cross_entropy(preds,labels)
    #------------要将梯度归零 因为pytorch会累加梯度
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    total_correct += get_num_correct(preds,labels)
    print('where',count,'loss:',loss.item())
    count +=1
print("epoch:",0,"total_correct",total_correct," loss:",total_loss,)
total_correct / len(train_set)

where 0 loss: 7.072266101837158
where 1 loss: 6.796487808227539
where 2 loss: 6.318027973175049
where 3 loss: 6.179681777954102
where 4 loss: 5.475547790527344
where 5 loss: 5.175508499145508
where 6 loss: 4.6894683837890625
where 7 loss: 4.41381311416626
where 8 loss: 4.40790319442749
where 9 loss: 4.095616817474365
where 10 loss: 3.798508644104004
where 11 loss: 3.4115304946899414
where 12 loss: 3.1555874347686768
where 13 loss: 3.1702916622161865
where 14 loss: 2.6597187519073486
where 15 loss: 2.8451459407806396
where 16 loss: 2.7532095909118652
where 17 loss: 2.4439148902893066
where 18 loss: 2.5306384563446045
where 19 loss: 2.500487804412842
where 20 loss: 2.3633148670196533
where 21 loss: 2.2519185543060303
where 22 loss: 2.3565421104431152
where 23 loss: 2.4086804389953613
where 24 loss: 2.3906443119049072
where 25 loss: 2.2535793781280518
where 26 loss: 2.248709201812744
where 27 loss: 2.273869514465332
where 28 loss: 2.2095186710357666
where 29 loss: 2.2107577323913574
where

0.30616

In [6]:
network =  resnet34().to('cuda')

train_loader = torch.utils.data.DataLoader(train_set,batch_size=100)
optimizer = optim.Adam(network.parameters(),lr = 0.0001)

total_loss = 0
total_correct = 0

for epoch in range(5):
    total_loss = 0
    total_correct = 0
    for batch in train_loader:
        images = batch[0].to('cuda')
        labels = batch[1].to('cuda')
        
        preds = network(images)
        loss = F.cross_entropy(preds,labels)
        #------------要将梯度归零 因为pytorch会累加梯度
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += get_num_correct(preds,labels)
    print("epoch:",epoch,"total_correct",total_correct," loss:",total_loss)
total_correct/len(train_set)

epoch: 0 total_correct 18221  loss: 945.8663983345032
epoch: 1 total_correct 24252  loss: 716.9034006595612
epoch: 2 total_correct 27969  loss: 619.8336728215218
epoch: 3 total_correct 30339  loss: 553.833398103714
epoch: 4 total_correct 32261  loss: 505.9548798799515


0.64522

In [7]:
torch.save(network.state_dict(),'pt')